In [1]:
from two_step import two_step_overland
from ana_new import trans_ana
from math import sqrt
from farPoint import bcdPoints
import numpy as np
import matplotlib.pyplot as plt


i1 = 30.0
i2 = 60.0
i3 = 15.0
L = 100.0
n = 0.1
slope = 0.01
alpha = 1./n*sqrt(slope)
m = 5./3

ie2 = i2/3600./1000.
q2 = ie2*L
S2 = m/(m+1)*alpha/ie2*(ie2*L/alpha)**((m+1)/m)
steady_time = (ie2*L/alpha)**(1/m)/ie2

# Time of change(d)
d = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]

aa = trans_ana(i1, i2, 100.0, 201, 0.5, 0.1, 0.01)
aa.run()

aaOut = np.array(zip(aa.out_q[:, 1], aa.out_s[:, 1]))

ba = trans_ana(i2, i3, 100.0, 201, 0.5, 0.1, 0.01)
ba.run()

baOut = np.array(zip(ba.out_q[:, 1], ba.out_s[:, 1]))

aaOut[:, 0] = (aaOut[:, 0] - aaOut[0, 0])/(aaOut[-1, 0] - aaOut[0, 0])
aaOut[:, 1] = (aaOut[:, 1] - aaOut[0, 1])/(aaOut[-1, 1] - aaOut[0, 1])

baOut[:, 0] = (baOut[:, 0] - baOut[-1, 0])/(baOut[0, 0] - baOut[-1, 0])
baOut[:, 1] = (baOut[:, 1] - baOut[-1, 1])/(baOut[0, 1] - baOut[-1, 1])

plt.plot(aaOut[:, 0], aaOut[:, 1], 'k--', lw=2.0)
plt.plot(baOut[:, 0], baOut[:, 1], 'k--', lw=2.0)

for i in range(0, len(d)):
    ToC = d[i]*steady_time
    ca = two_step_overland(i1, i2, i3, 100.0, 201, 0.5, 0.1, 0.01)
    ca.ie1_duration(ToC)
    ca.transition()
    
    SDC = zip(ca.q_curve[:, 0], ca.q_curve[:, 1], ca.s_curve[:, 1])
    SDC = np.array(SDC)
    
    re_qB = (SDC[0, 1] - aa.out_q[0, 1])/(aa.out_q[-1, 1] - aa.out_q[0, 1])
    re_sB = (SDC[0, 2] - aa.out_s[0, 1])/(aa.out_s[-1, 1] - aa.out_s[0, 1])
    re_qD = (SDC[-1, 1] - ba.out_q[-1, 1])/(ba.out_q[0, 1] - ba.out_q[-1, 1])
    re_sD = (SDC[-1, 2] - ba.out_s[-1, 1])/(ba.out_s[0, 1] - ba.out_s[-1, 1])
    
    # Normalize to 0~1
    if SDC[0, 1] > SDC[-1, 1]:
        SDC[:, 1] = (SDC[:, 1] - SDC[0, 1])/(SDC[-1, 1] - SDC[0, 1])
        SDC[:, 2] = (SDC[:, 2] - SDC[0, 2])/(SDC[-1, 2] - SDC[0, 2])
    else:
        SDC[:, 1] = (SDC[:, 1] - SDC[-1, 1])/(SDC[0, 1] - SDC[-1, 1])
        SDC[:, 2] = (SDC[:, 2] - SDC[-1, 2])/(SDC[0, 2] - SDC[-1, 2])
    
    # Compress to realm in 0~1 with Primary Curve
    SDC[:, 1] = SDC[:, 1]*(re_qD - re_qB) + re_qB
    SDC[:, 2] = SDC[:, 2]*(re_sD - re_sB) + re_sB

    plt.plot(SDC[:, 1], SDC[:, 2], color='dodgerblue')
    
i3 = 35.0
ab = trans_ana(i1, i2, 100.0, 201, 0.5, 0.1, 0.01)
ab.run()
abOut = np.array(zip(ab.out_q[:, 1], ab.out_s[:, 1]))

bb = trans_ana(i2, i3, 100.0, 201, 0.5, 0.1, 0.01)
bb.run()
bbOut = np.array(zip(bb.out_q[:, 1], bb.out_s[:, 1]))

abOut[:, 0] = (abOut[:, 0] - abOut[0, 0])/(abOut[-1, 0] - abOut[0, 0])
abOut[:, 1] = (abOut[:, 1] - abOut[0, 1])/(abOut[-1, 1] - abOut[0, 1])

bbOut[:, 0] = (bbOut[:, 0] - bbOut[-1, 0])/(bbOut[0, 0] - bbOut[-1, 0])
bbOut[:, 1] = (bbOut[:, 1] - bbOut[-1, 1])/(bbOut[0, 1] - bbOut[-1, 1])

plt.plot(abOut[:, 0], abOut[:, 1], 'k--', lw=2.0)
plt.plot(bbOut[:, 0], bbOut[:, 1], 'k--', lw=2.0)

for i in range(0, len(d)):
    ToC = d[i]*steady_time
    cb = two_step_overland(i1, i2, i3, 100.0, 201, 0.5, 0.1, 0.01)
    cb.ie1_duration(ToC)
    cb.run()
    
    SDC = bcdPoints(cb)[0]
    SDC = np.array(SDC)
    
    re_qB = (SDC[0, 1] - ab.out_q[0, 1])/(ab.out_q[-1, 1] - ab.out_q[0, 1])
    re_sB = (SDC[0, 2] - ab.out_s[0, 1])/(ab.out_s[-1, 1] - ab.out_s[0, 1])
    re_qD = (SDC[-1, 1] - bb.out_q[-1, 1])/(bb.out_q[0, 1] - bb.out_q[-1, 1])
    re_sD = (SDC[-1, 2] - bb.out_s[-1, 1])/(bb.out_s[0, 1] - bb.out_s[-1, 1])
    
    # Normalize to 0~1
    if SDC[0, 1] > SDC[-1, 1]:
        SDC[:, 1] = (SDC[:, 1] - SDC[0, 1])/(SDC[-1, 1] - SDC[0, 1])
        SDC[:, 2] = (SDC[:, 2] - SDC[0, 2])/(SDC[-1, 2] - SDC[0, 2])
    else:
        SDC[:, 1] = (SDC[:, 1] - SDC[-1, 1])/(SDC[0, 1] - SDC[-1, 1])
        SDC[:, 2] = (SDC[:, 2] - SDC[-1, 2])/(SDC[0, 2] - SDC[-1, 2])
    
    # Compress to realm in 0~1 with Primary Curve
    SDC[:, 1] = SDC[:, 1]*(re_qD - re_qB) + re_qB
    SDC[:, 2] = SDC[:, 2]*(re_sD - re_sB) + re_sB

    plt.plot(SDC[:, 1], SDC[:, 2], color='crimson')

plt.title('30 - 60 - 15 \n30 - 60- 35')
plt.savefig('SecondaryCurveNormalize_2.png', dpi=150)

(1.0, 0.0006524779401948101)
(2.0, 0.002071488337302571)
(3.0, 0.004071626424892357)
(4.0, 0.006576565531547916)
(5.0, 0.009539285354611094)
(6.0, 0.012926608140191295)
(7.0, 0.01671331376553227)
(8.0, 0.020879294086233926)
(9.0, 0.025407972779787756)
(10.0, 0.03028534321386898)
(11.0, 0.035499342416937606)
(12.0, 0.041039422720240706)
(13.0, 0.046896246828991774)
(14.0, 0.053061463706561035)
(15.0, 0.05952753944880745)
(16.0, 0.06628762679368229)
(17.0, 0.07333546251852882)
(18.0, 0.08066528543802894)
(19.0, 0.08827176992756795)
(20.0, 0.09614997135382719)
(21.0, 0.10429528077982314)
(22.0, 0.11270338699365215)
(23.0, 0.12137024439204612)
(24.0, 0.13029204559655544)
(25.0, 0.13946519793378526)
(26.0, 0.1488863030994038)
(27.0, 0.1585521394673389)
(28.0, 0.16845964661353485)
(29.0, 0.17860591170682374)
(30.0, 0.18898815748423087)
(31.0, 0.19960373157894182)
(32.0, 0.21045009700953335)
(33.0, 0.2215248236713566)
(34.0, 0.23282558069697423)
(35.0, 0.2443501295736682)
(36.0, 0.25609631792

ana_new.py:210: RuntimeWarning: invalid value encountered in double_scalars
  + alpha/ie1*ih**m)
ana_new.py:212: RuntimeWarning: invalid value encountered in double_scalars
  - alpha/ie1*m*(ih + ie1*t)**m-1)


(56.0, 0.5348260404970328)
(57.0, 0.5508380419754809)
(58.0, 0.5670384242623028)
(59.0, 0.5834260983537669)
(60.0, 0.5999999999999998)
(61.0, 0.6167590887367526)
(62.0, 0.6337023469703806)
(63.0, 0.6508287791122992)
(64.0, 0.6681374107594857)
(65.0, 0.685627287917904)
(66.0, 0.7032974762659628)
(67.0, 0.7211470604553722)
(68.0, 0.7391751434469677)
(69.0, 0.7573808458792537)
(70.0, 0.7757633054676136)
(71.0, 0.7943216764322647)
(72.0, 0.8130551289532083)
(73.0, 0.8319628486505294)
(74.0, 0.8510440360885462)
(75.0, 0.8702979063023957)
(76.0, 0.889723688345761)
(77.0, 0.9093206248585233)
(78.0, 0.9290879716532171)
(79.0, 0.949024997319235)
(80.0, 0.9691309828438076)
(81.0, 0.989405221248843)
(82.0, 1.0098470172427778)
(83.0, 1.030455686886637)
(84.0, 1.051230557273563)
(85.0, 1.0721709662211154)
(86.0, 1.0932762619756826)
(87.0, 1.1145458029284023)
(88.0, 1.1359789573420036)
(89.0, 1.1575751030880401)
(90.0, 1.179333627394003)
(91.0, 1.201253926599834)
(92.0, 1.223335405923393)
(93.0, 1.2

(368.0, 12.330448232036577)
(369.0, 12.386343221833261)
(370.0, 12.442339287615896)
(371.0, 12.498436338242733)
(372.0, 12.554634282899867)
(373.0, 12.610933031099178)
(374.0, 12.667332492676305)
(375.0, 12.723832577788622)
(376.0, 12.780433196913213)
(377.0, 12.837134260844916)
(378.0, 12.893935680694335)
(379.0, 12.950837367885892)
(380.0, 13.007839234155906)
(381.0, 13.064941191550652)
(382.0, 13.122143152424485)
(383.0, 13.179445029437945)
(384.0, 13.23684673555589)
(385.0, 13.294348184045647)
(386.0, 13.351949288475183)
(387.0, 13.409649962711269)
(388.0, 13.467450120917698)
(389.0, 13.52534967755349)
(390.0, 13.583348547371099)
(391.0, 13.641446645414693)
(392.0, 13.699643887018384)
(393.0, 13.757940187804515)
(394.0, 13.816335463681932)
(395.0, 13.874829630844305)
(396.0, 13.933422605768428)
(397.0, 13.992114305212562)
(398.0, 14.050904646214763)
(399.0, 14.109793546091257)
(400.0, 14.168780922434811)
(401.0, 14.227866693113102)
(402.0, 14.28705077626713)
(403.0, 14.346333090309

(664.0, 32.97458198037239)
(665.0, 33.057391048436585)
(666.0, 33.140283174757485)
(667.0, 33.223258317743564)
(668.0, 33.306316435886444)
(669.0, 33.389457487760666)
(670.0, 33.472681432023364)
(671.0, 33.555988227413984)
(672.0, 33.63937783275402)
(673.0, 33.72285020694668)
(674.0, 33.806405308976665)
(675.0, 33.89004309790983)
(676.0, 33.97376353289292)
(677.0, 34.05756657315333)
(678.0, 34.14145217799878)
(679.0, 34.22542030681704)
(680.0, 34.30947091907569)
(681.0, 34.39360397432182)
(682.0, 34.477819432181754)
(683.0, 34.56211725236078)
(684.0, 34.6464973946429)
(685.0, 34.730959818890554)
(686.0, 34.81550448504433)
(687.0, 34.90013135312272)
(688.0, 34.98484038322185)
(689.0, 35.06963153551523)
(690.0, 35.15450477025346)
(691.0, 35.239460047763984)
(692.0, 35.32449732845088)
(693.0, 35.409616572794484)
(694.0, 35.49481774135124)
(695.0, 35.58010079475344)
(696.0, 35.66546569370888)
(697.0, 35.75091239900071)
(698.0, 35.83644087148711)
(699.0, 35.92205107210107)
(700.0, 36.007742

...

(378.0, 9.001813869406304)
(379.0, 9.041539396944067)
(380.0, 9.081334863826974)
(381.0, 9.121200208650738)
(382.0, 9.161135370226134)
(383.0, 9.201140287577715)
(384.0, 9.241214899942484)
(385.0, 9.281359146768617)
(386.0, 9.321572967714168)
(387.0, 9.361856302645815)
(388.0, 9.402209091637586)
(389.0, 9.44263127496962)
(390.0, 9.483122793126933)
(391.0, 9.52368358679818)
(392.0, 9.564313596874443)
(393.0, 9.605012764448032)
(394.0, 9.645781030811282)
(395.0, 9.686618337455366)
(396.0, 9.727524626069126)
(397.0, 9.768499838537904)
(398.0, 9.809543916942385)
(399.0, 9.85065680355745)
(400.0, 9.891838440851044)
(401.0, 9.933088771483044)
(402.0, 9.974407738304146)
(403.0, 10.015795284354754)
(404.0, 10.057251352863876)
(405.0, 10.09877588724804)
(406.0, 10.140368831110218)
(407.0, 10.182030128238733)
(408.0, 10.223759722606225)
(409.0, 10.265557558368565)
(410.0, 10.307423579863837)
(411.0, 10.349357731611281)
(412.0, 10.391359958310264)
(413.0, 10.433430204839283)
(414.0, 10.4755684162

(680.0, 23.952924756229)
(681.0, 24.01166167894563)
(682.0, 24.070456130489)
(683.0, 24.129308082727636)
(684.0, 24.188217507585012)
(685.0, 24.247184377039353)
(686.0, 24.306208663123464)
(687.0, 24.365290337924513)
(688.0, 24.424429373583894)
(689.0, 24.483625742297004)
(690.0, 24.542879416313063)
(691.0, 24.60219036793497)
(692.0, 24.66155856951906)
(693.0, 24.72098399347498)
(694.0, 24.780466612265474)
(695.0, 24.84000639840623)
(696.0, 24.89960332446568)
(697.0, 24.959257363064822)
(698.0, 25.01896848687708)
(699.0, 25.078736668628075)
(700.0, 25.138561881095505)
(701.0, 25.198444097108915)
(702.0, 25.258383289549585)
(703.0, 25.318379431350312)
(704.0, 25.378432495495243)
(705.0, 25.438542455019732)
(706.0, 25.49870928301015)
(707.0, 25.55893295260373)
(708.0, 25.619213436988353)
(709.0, 25.67955070940247)
(710.0, 25.73994474313485)
(711.0, 25.80039551152444)
(712.0, 25.860902987960234)
(713.0, 25.921467145881074)
(714.0, 25.982087958775487)
(715.0, 26.042765400181526)
(716.0, 26

(987.0, 44.569621935362655)
(988.0, 44.64490844745476)
(989.0, 44.72024577731114)
(990.0, 44.79563390779839)
(991.0, 44.87107282180615)
(992.0, 44.94656250224707)
(993.0, 45.02210293205682)
(994.0, 45.097694094193955)
(995.0, 45.17333597163993)
(996.0, 45.24902854739897)
(997.0, 45.3247718044981)
(998.0, 45.40056572598703)
(999.0, 45.476410294938105)
(1000.0, 45.55230549444632)
(1001.0, 45.62825130762918)
(1002.0, 45.70424771762671)
(1003.0, 45.78029470760133)
(1004.0, 45.856392260737906)
(1005.0, 45.93254036024363)
(1006.0, 46.00873898934794)
(1007.0, 46.08498813130255)
(1008.0, 46.16128776938136)
(1009.0, 46.237637886880385)
(1010.0, 46.314038467117705)
(1011.0, 46.39048949343345)
(1012.0, 46.46699094918976)
(1013.0, 46.54354281777062)
(1014.0, 46.62014508258196)
(1015.0, 46.696797727051546)
(1016.0, 46.773500734628875)
(1017.0, 46.85025408878518)
(1018.0, 46.92705777301338)
(1019.0, 47.003911770828054)
(1020.0, 47.08081606576529)
(1021.0, 47.15777064138276)
(1022.0, 47.2347754812596

(1289.0, 69.54489001802631)
(1290.0, 69.63483425472823)
(1291.0, 69.72482498625604)
(1292.0, 69.81486220060178)
(1293.0, 69.90494588576989)
(1294.0, 69.9950760297772)
(1295.0, 70.08525262065282)
(1296.0, 70.17547564643824)
(1297.0, 70.2657450951873)
(1298.0, 70.35606095496603)
(1299.0, 70.44642321385275)
(1300.0, 70.53683185993808)
(1301.0, 70.6272868813248)
(1302.0, 70.71778826612787)
(1303.0, 70.80833600247449)
(1304.0, 70.89893007850398)
(1305.0, 70.98957048236775)
(1306.0, 71.08025720222939)
(1307.0, 71.17099022626452)
(1308.0, 71.26176954266089)
(1309.0, 71.35259513961819)
(1310.0, 71.44346700534824)
(1311.0, 71.53438512807482)
(1312.0, 71.62534949603364)
(1313.0, 71.71636009747246)
(1314.0, 71.80741692065091)
(1315.0, 71.8985199538406)
(1316.0, 71.98966918532493)
(1317.0, 72.08086460339928)
(1318.0, 72.17210619637083)
(1319.0, 72.2633939525586)
(1320.0, 72.35472786029344)
(1321.0, 72.44610790791793)
(1322.0, 72.53753408378653)
(1323.0, 72.62900637626531)
(1324.0, 72.7205247737321

(1603.0, 100.01525186996746)
ie0=30.00
ie1=60.00
ie2=35.00
ToC= 12.92
ie0=30.00
ie1=60.00
ie2=35.00
ToC=129.20
ie0=30.00
ie1=60.00
ie2=35.00
ToC=258.40
ie0=30.00
ie1=60.00
ie2=35.00
ToC=387.60
ie0=30.00
ie1=60.00
ie2=35.00
ToC=516.80
ie0=30.00
ie1=60.00
ie2=35.00
ToC=646.00
ie0=30.00
ie1=60.00
ie2=35.00
ToC=775.20
ie0=30.00
ie1=60.00
ie2=35.00
ToC=904.40
ie0=30.00
ie1=60.00
ie2=35.00
ToC=1033.60
ie0=30.00
ie1=60.00
ie2=35.00
ToC=1162.79
ie0=30.00
ie1=60.00
ie2=35.00
ToC=1201.55
